## Chat Completion agent

[MS Learn sample](https://learn.microsoft.com/en-us/semantic-kernel/frameworks/agent/chat-completion-agent?pivots=programming-language-python)

In [15]:
import asyncio
import os
import sys
from dotenv import load_dotenv
from datetime import datetime

from semantic_kernel.agents import ChatCompletionAgent
from semantic_kernel.connectors.ai.function_choice_behavior import FunctionChoiceBehavior
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.contents.chat_history import ChatHistory
from semantic_kernel.contents.chat_message_content import ChatMessageContent
from semantic_kernel.contents.utils.author_role import AuthorRole
from semantic_kernel.kernel import Kernel

load_dotenv()

True

In [16]:
chatServiceName = "chatService"
advocateName = "Advocate"
prosecutorName = "Prosecutor"
advocateInstructions = """You are the advocate agent supporting the user proposal. You should articulate reasons supporting that proposal.
    You should also argue against any arguments raised by the prosecutor agent and listed in the history.
    Be brief and concise in presenting your arguments.
    If you cannot find a good argument respond with 'I give up.'"""
prosecutorInstructions = """You are the Prosecutor agent whose job is to argue against the User proposal and against the Advocate agents arguments.
    Be brief and concise in presenting your arguments.
    If you cannot find a good argument respond with 'I give up.'"""

In [17]:
chat_completion_service = AzureChatCompletion(
    deployment_name=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),  
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    service_id=chatServiceName, # Optional; for targeting specific services within Semantic Kernel
)

In [18]:
# Define the Kernel
kernel = Kernel()

# Add the AzureChatCompletion AI Service to the Kernel
kernel.add_service(AzureChatCompletion(service_id="chatService"))

# Create the agent
advocate = ChatCompletionAgent(
  service_id=chatServiceName, 
  kernel=kernel, 
  name=advocateName, 
  instructions=advocateInstructions,
)
prosecutor = ChatCompletionAgent(
  service_id=chatServiceName, 
  kernel=kernel, 
  name=prosecutorName, 
  instructions=prosecutorInstructions,
)

In [20]:
history = ChatHistory()
user_input = input("User:> ")
print(f"##Proposal\n\n{user_input}")
turns = 0
if user_input:
    history.add_message(ChatMessageContent(role=AuthorRole.USER, content=f"User proposal: {user_input}"))
    is_complete: bool = False
    agents = [ advocate, prosecutor ]
    agentNo = 0
    while not is_complete:
        agent = agents[agentNo]
        async for response in agent.invoke(history=history):
            print(f"##{agent.name}\n\n {response.content}")
            history.add_message(ChatMessageContent(role=AuthorRole.ASSISTANT, content=f"{agent.name}: {response.content}"))
        if response.content == "I give up.":
            is_complete = True
            break
        turns += 1
        if turns == 5:
            print("##Timeout!!!")
            is_complete = True
            break
        agentNo = (agentNo + 1) % 2


##Proposal

Goal justifies the means
##Advocate

 The principle that the goal justifies the means is rooted in the idea that achieving a significant outcome can outweigh the methods employed to get there. Here are several reasons supporting this proposal:

1. **Pragmatism**: In many cases, the urgency and importance of a goal (such as saving lives or achieving social justice) can necessitate unconventional methods. If the end result is positive and beneficial, it can be argued that the means were justified.

2. **Historical Precedents**: History is filled with examples where disregarding strict ethical norms led to substantial progress (e.g., social movements that challenged unjust laws). These examples showcase that when the ultimate goal is altruistic, the means used can be viewed in a different light.

3. **Moral Flexibility**: Rigid adherence to moral codes can sometimes prevent necessary change. By accepting that goals can justify means, society allows for flexibility in thought a